In [1]:
!pwd


/Users/vascomeerman/Coding/Harvard/CS215/Project/Style-Transfer-App


In [2]:
MODEL_PATH = "./persistent-folder/st_models/"

STYLES = {
    "candy": "candy",
    "composition 6": "composition_vii",
    "feathers": "feathers",
    "la_muse": "la_muse",
    "mosaic": "mosaic",
    "starry night": "starry_night",
    "the scream": "the_scream",
    "the wave": "the_wave",
    "udnie": "udnie",
}

In [3]:
# backend/inference.py

import cv2


def inference(model, image):
    model_name = f"{MODEL_PATH}{model}.t7"
    model = cv2.dnn.readNetFromTorch(model_name)

    height, width = int(image.shape[0]), int(image.shape[1])
    new_width = int((640 / height) * width)
    resized_image = cv2.resize(image, (new_width, 640), interpolation=cv2.INTER_AREA)

    # Create our blob from the image
    # Then perform a forward pass run of the network
    # The Mean values for the ImageNet training set are R=103.93, G=116.77, B=123.68

    inp_blob = cv2.dnn.blobFromImage(
        resized_image,
        1.0,
        (new_width, 640),
        (103.93, 116.77, 123.68),
        swapRB=False,
        crop=False,
    )

    model.setInput(inp_blob)
    output = model.forward()

    # Reshape the output Tensor,
    # add back the mean substruction,
    # re-order the channels
    output = output.reshape(3, output.shape[2], output.shape[3])
    output[0] += 103.93
    output[1] += 116.77
    output[2] += 123.68

    output = output.transpose(1, 2, 0)
    return output, resized_image

In [17]:
import numpy as np
from PIL import Image
import base64


In [20]:

img_path = 'data/portraits/dongyun.jpg'
style = 'mosaic'

image = np.array(Image.open(img_path))
model = STYLES[style]

output, resized = inference(model, image)
name = f"./persistent-folder/st_models/test.jpg"
output.shape
cv2.imwrite(name, output)
# np_img = np.squeeze(output, axis=2)
# 
# Image.fromarray(np_img)
# Image.fromarray((output * 255).astype(np.uint8))
   # Read image to encoded bytes for response
with open(name, "rb") as image_file:
    encoded_image_string = base64.b64encode(image_file.read())
    
    
encoded_image_string 

b'...'